# Association Rules - 연관규칙

# 1. Read Data_Set and Preprocessing
> ## 1) Read ' order.tsv'

- df.info( )

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/rusita-ai/pyData/master/orders.tsv'
df = pd.read_table(url)
df.info()
# .tsv -> tab으로 구분

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [22]:
df.head(15)
# order_id : 주문번호
# quantity : 개수
# item_name : 음식명
# choice_description : 선택사항
# item_price : 음식 가격

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


> ## 2) 데이터 정보 확인

- df_1 지정 후 'order_id' 및 'item_name' 종류 확인
- 한 개인 'order_id'가 여러 개인 'item_name'으로 분리되어 지정
   - 'item_name' 하나 당 'order_id' 하나로 

- 비정형 데이터를 정형데이터로 바꾸는 3단계 중 1단계 : 필요한 변수 따로 떼내 리스트 만들고, 중복 제거 

In [3]:
df_1 = df[['order_id', 'item_name']]
df_1.order_id.unique().shape, df_1.item_name.unique().shape
# order_id = 주문 1834개
# item_name = 주문된 음식 50종류
# .unique() : 중복제외

((1834,), (50,))

- 'order_id' 1834개

In [4]:
order_ID = list(df_1.order_id.unique())
order_ID[:10], order_ID[-10:]

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834])

- 'item_name' 50종류

In [6]:
item_NAME = list(df_1.item_name.unique())
item_NAME[:5], item_NAME[-5:]

(['Chips and Fresh Tomato Salsa',
  'Izze',
  'Nantucket Nectar',
  'Chips and Tomatillo-Green Chili Salsa',
  'Chicken Bowl'],
 ['Barbacoa Salad Bowl',
  'Salad',
  'Veggie Crispy Tacos',
  'Veggie Salad',
  'Carnitas Salad'])

> ## 3) Preprocessing

- 1835 길이의 2차원 리스트 생성
   - 파이썬은 0번째부터 세기 때문에 필요한 것보다 더 크게 만들어서 나중에 삭제

- 2단계 : 중복 제거된 두 변수를 넣을 2차원 리스트 만들기

In [7]:
orderItems = [[] for i in range(1835)]
len(orderItems)
# order_ID를 담기 위해 빈 리스트 생성

1835

- 'order_id' 별로 'item_name' 묶기

In [9]:
num = 0
for i in df_1.item_name :
  orderItems[df_1.order_id[num]].append(i)
  num = num + 1 
# order_id = 1에는 음식 4개 주문, 2에는 1개, 3에는 2개 ...
# 위에서 만든 빈 리스트에 order_id 1~1834번에 매칭되는 item_name(주문음식) 채워넣기

In [10]:
orderItems[:5], orderItems[-5:]

([[],
  ['Chips and Fresh Tomato Salsa',
   'Izze',
   'Nantucket Nectar',
   'Chips and Tomatillo-Green Chili Salsa'],
  ['Chicken Bowl'],
  ['Chicken Bowl', 'Side of Chips'],
  ['Steak Burrito', 'Steak Soft Tacos']],
 [['Steak Burrito', 'Veggie Burrito'],
  ['Carnitas Bowl', 'Chips', 'Bottled Water'],
  ['Chicken Soft Tacos', 'Chips and Guacamole'],
  ['Steak Burrito', 'Steak Burrito'],
  ['Chicken Salad Bowl', 'Chicken Salad Bowl', 'Chicken Salad Bowl']])

- 첫 번째 빈 리스트 제거 및 중복 아이템 단일화

In [13]:
orderItems.pop(0)
num = 0
for i in orderItems :
  orderItems[num] = list(set(orderItems[num]))
  num = num + 1
# orderItems.pop(0) : 비어있는 첫번째 인덱스(0) 삭제

In [14]:
orderItems[:5], orderItems[-5:]

([['Izze',
   'Chips and Tomatillo-Green Chili Salsa',
   'Nantucket Nectar',
   'Chips and Fresh Tomato Salsa'],
  ['Chicken Bowl'],
  ['Chicken Bowl', 'Side of Chips'],
  ['Steak Soft Tacos', 'Steak Burrito'],
  ['Chips and Guacamole', 'Steak Burrito']],
 [['Steak Burrito', 'Veggie Burrito'],
  ['Bottled Water', 'Carnitas Bowl', 'Chips'],
  ['Chips and Guacamole', 'Chicken Soft Tacos'],
  ['Steak Burrito'],
  ['Chicken Salad Bowl']])

# 2. TransactionEncoder( )

- 3단계 : mlxtend, pandas를 이용해 변환

In [16]:
from mlxtend.preprocessing import TransactionEncoder
TSE = TransactionEncoder()
Transac_array = TSE.fit_transform(orderItems)
# orderItems처럼 list형태가 들어가야 한다. -> array로 변경

- pandas DataFrame 구조 변환

In [17]:
order_df = pd.DataFrame(Transac_array, columns = TSE.columns_)
order_df.head()
# array -> DF
# 행 : order_id(주문목록)가 boolen형태로 들어가고,
# 열 : 50개의 item_name(주문음식명)이 들어간다.
# ex) ChickenBowl은 2,3번 주문에 들어가(True) 있다.

,6 Pack Soft Drink,Barbacoa Bowl,Barbacoa Burrito,Barbacoa Crispy Tacos,Barbacoa Salad Bowl,Barbacoa Soft Tacos,Bottled Water,Bowl,Burrito,Canned Soda,Canned Soft Drink,Carnitas Bowl,Carnitas Burrito,Carnitas Crispy Tacos,Carnitas Salad,Carnitas Salad Bowl,Carnitas Soft Tacos,Chicken Bowl,Chicken Burrito,Chicken Crispy Tacos,Chicken Salad,Chicken Salad Bowl,Chicken Soft Tacos,Chips,Chips and Fresh Tomato Salsa,Chips and Guacamole,Chips and Mild Fresh Tomato Salsa,Chips and Roasted Chili Corn Salsa,Chips and Roasted Chili-Corn Salsa,Chips and Tomatillo Green Chili Salsa,Chips and Tomatillo Red Chili Salsa,Chips and Tomatillo-Green Chili Salsa,Chips and Tomatillo-Red Chili Salsa,Crispy Tacos,Izze,Nantucket Nectar,Salad,Side of Chips,Steak Bowl,Steak Burrito,Steak Crispy Tacos,Steak Salad,Steak Salad Bowl,Steak Soft Tacos,Veggie Bowl,Veggie Burrito,Veggie Crispy Tacos,Veggie Salad,Veggie Salad Bowl,Veggie Soft Tacos
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [24]:
order_df.shape

(1834, 50)

# 3. apropri( )

- 지지도(support) 0.05 이상인 주문 추출
- use_colnames : item_name으로 출력
- max_len : 주문의 최대 길이 지정

In [18]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(order_df,
                            min_support = 0.05,
                            use_colnames = True,
                            max_len = None)
frequent_itemsets

,support,itemsets
0,0.083969,(Bottled Water)
1,0.051254,(Canned Soda)
2,0.150491,(Canned Soft Drink)
3,0.335333,(Chicken Bowl)
4,0.266630,(Chicken Burrito)
5,0.053435,(Chicken Salad Bowl)
6,0.058342,(Chicken Soft Tacos)
7,0.113413,(Chips)
8,0.059978,(Chips and Fresh Tomato Salsa)
9,0.258451,(Chips and Guacamole)


# 4. association_rules( )

> ## 1) 지지도(support)가 최소 0.05 이상인 연관관계 출력

- antecedents(조건절) -> consequents(결과절)
- 전체 주문 중 조건절과 결과절을 포함한 비율
- 방향성 없음
- A와 B가 동시에 발생할 확률

In [19]:
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets,
                  metric = 'support',
                  min_threshold = 0.05)
# ex) ChickenBowl을 구매할 p = 0.335333, CannedSoftDrink를 구매할 p = 0.150491,둘을 동시에 구매할 p = 0.060523, 
# ChickenBowl에 대한 CannedSoftDrinkp의 신뢰p = 0.180488, 둘은 lift > 1이므로 연관성 있음 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Chicken Bowl),(Canned Soft Drink),0.335333,0.150491,0.060523,0.180488,1.199328,0.010059,1.036604
1,(Canned Soft Drink),(Chicken Bowl),0.150491,0.335333,0.060523,0.402174,1.199328,0.010059,1.111807
2,(Chicken Bowl),(Chips),0.335333,0.113413,0.066521,0.198374,1.749124,0.028490,1.105985
3,(Chips),(Chicken Bowl),0.113413,0.335333,0.066521,0.586538,1.749124,0.028490,1.607568
4,(Chips and Guacamole),(Chicken Bowl),0.258451,0.335333,0.081243,0.314346,0.937416,-0.005424,0.969392
5,(Chicken Bowl),(Chips and Guacamole),0.335333,0.258451,0.081243,0.242276,0.937416,-0.005424,0.978653
6,(Chips and Guacamole),(Chicken Burrito),0.258451,0.266630,0.062159,0.240506,0.902022,-0.006752,0.965603
7,(Chicken Burrito),(Chips and Guacamole),0.266630,0.258451,0.062159,0.233129,0.902022,-0.006752,0.966979


> ## 2) 신뢰도(confidence)가 최소 0.3 이상인 연관관계 출력

- 조건절이 있을 때 결과절도 있는 비율
- 조건부 확률
- 방향성 존재

In [20]:
association_rules(frequent_itemsets,
                  metric = 'confidence',
                  min_threshold = 0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Canned Soft Drink),(Chicken Bowl),0.150491,0.335333,0.060523,0.402174,1.199328,0.010059,1.111807
1,(Chips),(Chicken Bowl),0.113413,0.335333,0.066521,0.586538,1.749124,0.028490,1.607568
2,(Chips and Guacamole),(Chicken Bowl),0.258451,0.335333,0.081243,0.314346,0.937416,-0.005424,0.969392


> ## 3) 향상도(lift)가 최소 0.1 이상인 연관관계 출력

- 향상도가 1이라면 조건절과 결과절은 독립관계
- 1보다 크거나 작다면 우연이 아닌 필연적 관계

In [21]:
association_rules(frequent_itemsets,
                  metric = 'lift',
                  min_threshold = 0.1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Chicken Bowl),(Canned Soft Drink),0.335333,0.150491,0.060523,0.180488,1.199328,0.010059,1.036604
1,(Canned Soft Drink),(Chicken Bowl),0.150491,0.335333,0.060523,0.402174,1.199328,0.010059,1.111807
2,(Chicken Bowl),(Chips),0.335333,0.113413,0.066521,0.198374,1.749124,0.028490,1.105985
3,(Chips),(Chicken Bowl),0.113413,0.335333,0.066521,0.586538,1.749124,0.028490,1.607568
4,(Chips and Guacamole),(Chicken Bowl),0.258451,0.335333,0.081243,0.314346,0.937416,-0.005424,0.969392
5,(Chicken Bowl),(Chips and Guacamole),0.335333,0.258451,0.081243,0.242276,0.937416,-0.005424,0.978653
6,(Chips and Guacamole),(Chicken Burrito),0.258451,0.266630,0.062159,0.240506,0.902022,-0.006752,0.965603
7,(Chicken Burrito),(Chips and Guacamole),0.266630,0.258451,0.062159,0.233129,0.902022,-0.006752,0.966979


### 
# The End
###